In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier as RF

In [3]:
data = pd.read_csv('data/train.csv')

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.4+ MB


In [61]:
data.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,...,0,0,0,0,0,0,0,0,0,8
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,...,0,0,0,0,0,0,0,0,0,4
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,...,0,0,0,0,0,0,0,0,0,8
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,...,0,0,0,0,0,0,0,0,0,8
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,...,0,0,0,0,0,0,0,0,0,8


#Identify null columns

In [62]:
null_cols = []
for col in data.columns:
    s = data[col].isnull().sum()
    print col, s
    if s > 0:
        null_cols.append(col)

Id 0
Product_Info_1 0
Product_Info_2 0
Product_Info_3 0
Product_Info_4 0
Product_Info_5 0
Product_Info_6 0
Product_Info_7 0
Ins_Age 0
Ht 0
Wt 0
BMI 0
Employment_Info_1 19
Employment_Info_2 0
Employment_Info_3 0
Employment_Info_4 6779
Employment_Info_5 0
Employment_Info_6 10854
InsuredInfo_1 0
InsuredInfo_2 0
InsuredInfo_3 0
InsuredInfo_4 0
InsuredInfo_5 0
InsuredInfo_6 0
InsuredInfo_7 0
Insurance_History_1 0
Insurance_History_2 0
Insurance_History_3 0
Insurance_History_4 0
Insurance_History_5 25396
Insurance_History_7 0
Insurance_History_8 0
Insurance_History_9 0
Family_Hist_1 0
Family_Hist_2 28656
Family_Hist_3 34241
Family_Hist_4 19184
Family_Hist_5 41811
Medical_History_1 8889
Medical_History_2 0
Medical_History_3 0
Medical_History_4 0
Medical_History_5 0
Medical_History_6 0
Medical_History_7 0
Medical_History_8 0
Medical_History_9 0
Medical_History_10 58824
Medical_History_11 0
Medical_History_12 0
Medical_History_13 0
Medical_History_14 0
Medical_History_15 44596
Medical_History_1

In [63]:
for col in null_cols:
    print '-----%s-----'% col
    print 'NULL distribution %d rows' % len(data[data[col].isnull()])
    print data[data[col].isnull()]['Response'].describe()
    print 'NOT NULL distribution %d rows' % len(data[~data[col].isnull()])
    print data[~data[col].isnull()]['Response'].describe()

-----Employment_Info_1-----
NULL distribution 19 rows
count    19.000000
mean      1.789474
std       1.873172
min       1.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       6.000000
Name: Response, dtype: float64
NOT NULL distribution 59362 rows
count    59362.000000
mean         5.638068
std          2.456045
min          1.000000
25%          4.000000
50%          6.000000
75%          8.000000
max          8.000000
Name: Response, dtype: float64
-----Employment_Info_4-----
NULL distribution 6779 rows
count    6779.000000
mean        5.539313
std         2.413517
min         1.000000
25%         4.000000
50%         6.000000
75%         8.000000
max         8.000000
Name: Response, dtype: float64
NOT NULL distribution 52602 rows
count    52602.000000
mean         5.649405
std          2.462102
min          1.000000
25%          4.000000
50%          6.000000
75%          8.000000
max          8.000000
Name: Response, dtype: float64
-----Employment_Info_6-----


In [64]:
data['Response'].describe()

count    59381.000000
mean         5.636837
std          2.456833
min          1.000000
25%          4.000000
50%          6.000000
75%          8.000000
max          8.000000
Name: Response, dtype: float64

In [65]:
categoricals = ['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 
                'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 
                'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 
                'InsuredInfo_6', 'InsuredInfo_7', 'Insurance_History_1', 'Insurance_History_2', 
                'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_7', 'Insurance_History_8', 
                'Insurance_History_9', 'Family_Hist_1', 'Medical_History_2', 'Medical_History_3', 
                'Medical_History_4', 'Medical_History_5', 'Medical_History_6', 'Medical_History_7', 
                'Medical_History_8', 'Medical_History_9', 'Medical_History_10', 'Medical_History_11', 
                'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 'Medical_History_16', 
                'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20', 
                'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 
                'Medical_History_26', 'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 
                'Medical_History_30', 'Medical_History_31', 'Medical_History_33', 'Medical_History_34', 
                'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 'Medical_History_38', 
                'Medical_History_39', 'Medical_History_40', 'Medical_History_41']
numericals = ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 
              'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 
              'Family_Hist_5']

#Convert categorical variables

In [66]:
dummies = pd.DataFrame({'id': [0] * len(data)})
for col in categoricals:
    dummy = pd.get_dummies(data[col])
    for i in xrange(dummy.shape[1]):
        val = dummy.columns.values[i]
        dummies['%s_%s' % (col, val)] = dummy[val]
dummies = dummies.drop('id', axis=1)

In [67]:
data_dummies = pd.concat((data, dummies), axis=1)
data_dummies = data_dummies.drop(categoricals, axis=1)

In [69]:
null_cols.remove('Medical_History_10')

#Find Feature Importances

In [74]:
non_nulldf = data_dummies.drop(null_cols + ['Id'], axis=1)
X = non_nulldf.drop('Response', axis=1)
y = non_nulldf['Response']
rf = RF(25).fit(X, y)

In [79]:
for i in np.argsort(rf.feature_importances_)[::-1]:
    print non_nulldf.columns[i]

BMI
Wt
Ins_Age
Product_Info_4
Ht
Medical_History_3_3
Medical_Keyword_15
Medical_History_4_1
Medical_History_22_2
InsuredInfo_3_7
InsuredInfo_3_2
Medical_History_23_2
Employment_Info_2_7
InsuredInfo_3_5
Medical_History_2_111
Medical_History_41_2
Medical_History_40_3
Product_Info_2_D2
Medical_History_9_1
Medical_History_2_161
Medical_History_8_3
Family_Hist_1_1
Medical_History_28_3
Family_Hist_1_2
Medical_History_29_2
InsuredInfo_6_1
Product_Info_5_3
Employment_Info_2_11
Product_Info_6_1
Medical_History_33_3
Medical_History_34_2
InsuredInfo_5_3
Medical_Keyword_3
Product_Info_2_D3
Medical_History_2_490
Employment_Info_5_3
Product_Info_2_C4
InsuredInfo_1_1
InsuredInfo_4_2
InsuredInfo_3_11
Product_Info_2_D1
Product_Info_2_A7
Insurance_History_8_2
Employment_Info_2_13
Product_Info_2_A6
Insurance_History_3_3
Medical_History_39_2
Insurance_History_7_3
Insurance_History_1_1
Medical_History_14_3
InsuredInfo_3_10
InsuredInfo_7_3
InsuredInfo_3_1
Medical_History_16_2
Medical_Keyword_25
Medical_Hist

#Impute variables with no info as NaN

In [91]:
for col in ['Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5',
            'Medical_History_24', 'Medical_History_32']:
    xtrain = non_nulldf[~data_dummies[col].isnull()]
    ytrain = data_dummies[~data_dummies[col].isnull()][col]
    xtest = non_nulldf[data_dummies[col].isnull()]
    rf = RF(25).fit(xtrain, ytrain)
    pred = rf.predict(xtest)
    data_dummies.loc[data_dummies[col].isnull(), col] = pred

In [4]:
for col in data.columns:
    if data[col].dtype == 'object':
        print col

Product_Info_2
